In [7]:
import re
import codecs
import pandas as pd
from flask import Flask, request

def decode(input_filepath, output_filepath):
    with codecs.open(input_filepath, "r") as input_file:
        with codecs.open(output_filepath, "w", encoding="utf-8") as output_file:
            for line in input_file:
                file_content = re.sub("\t", ",", line)
                output_file.write(file_content)

decode("pokemon2-8.tsv", "pokemon.csv")

def df_import(csv_file):
    poke_df = pd.read_csv(csv_file, index_col="english_name")
    return poke_df

df_import('pokemon.csv')

app = Flask(__name__)
poke_df = df_import('pokemon.csv')
app.config["db"] = poke_df

@app.route("/pokedex", methods=["GET"])
def pokedex_data():
    """
    HTTP GET: Query for Pokemon 
    """
    global poke_df
    name = request.args.get("name", default=None)
    if name is not None:
        result_df = poke_df.loc[poke_df["english_name"] == name]
    else:
        print("Please enter a valid Pokemon!")
    resp_json = {
        "query_name": name,
        "result": result_df.to_dict(orient="records"),
    }
    resp_headers = {
        "content-type": "application/json",
    }

    return resp_json, 200, resp_headers

if __name__ == '__main__':
    app.run('0.0.0.0', 8080)



 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://172.25.211.59:8080 (Press CTRL+C to quit)


In [8]:
# Battler Logic
def stat_grab(pokemon, stat):
    poke_df = df_import('pokemon.csv')
    stat = poke_df.at[pokemon, stat]
    return stat

poke1_attack = stat_grab('Bulbasaur', 'attack')
poke1_defense = stat_grab('Bulbasaur', 'defense')
poke1_speed = stat_grab('Bulbasaur', 'speed')

poke1_attack = stat_grab('Charmander', 'attack')
poke1_defense = stat_grab('Charmander', 'defense')
poke1_speed = stat_grab('Charmander', 'speed')

# poke_battle()

49